Implementar la función create_gauss_filter(h, w, k_size, sigma) para crear filtros gaussianos para 
filtrado espectral. Debe retornar un filtro gaussiano de tamaño HxW en dominio espacial y su transformada 
de Fourier.

1. Graficar ambas representaciones para diferentes tamaños de kernel y sigma. Aplicar el filtro una 
imagen para validar el funcionamiento en el dominio espectral. 
2. Usando el método descripto en el paper “Image Sharpness Measure for Blurred Images in Frequency 
Domain” comparar el resultado de un filtrado por convolución con el filtrado espectral.
3. Repetir la comparación usando uno de los métodos descriptos en el apéndice del paper “Analysis of 
focus measure operators in shape-from-focus”

In [1]:
%matplotlib

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: <object object at 0x000001F02ECC7540>


In [2]:
def create_gauss_filter(h, w, k_size, sigma):
    '''
    ¿¿¿Devuelve una imagen filtrada mediante filtro
    gaussiano espectral???
    O devuelve el filtro?
    No se bien que devuelve
    -------------------------------------------------------
    Parámetros:
    param h: alto de la imagen
    param w: ancho de la imagen
    param k_size: Tamaño del núcleo, positivo impar (3,5,...)
    param sigma:  Desvío estándar de la Gaussiana. Si no es positivo, se calcula como sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8 
    '''
    kernel_1d = cv.getGaussianKernel(ksize=k_size, sigma=sigma) 
    kernel_2d = kernel_1d * kernel_1d.T

    filter = kernel_2d

    return filter


In [4]:

create_gauss_filter(10,10,3,1)

array([[0.07511361, 0.1238414 , 0.07511361],
       [0.1238414 , 0.20417996, 0.1238414 ],
       [0.07511361, 0.1238414 , 0.07511361]])